<a href="https://colab.research.google.com/github/wsilveira-splunk/lz78_algorithm/blob/master/lz78.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

cd /content/

if [ ! -d "lz78_algorithm" ]; then
  git clone https://github.com/wsilveira-splunk/lz78_algorithm.git
fi

cd lz78_algorithm
git pull origin master
cd -

Already up to date.
/content


Cloning into 'lz78_algorithm'...
From https://github.com/wsilveira-splunk/lz78_algorithm
 * branch            master     -> FETCH_HEAD


In [2]:
import os

In [3]:
def lz78_compress(raw_text):

    dictionary = {'': 0}

    last_seq_letter = ''
    lz78_text = ''

    for letter in raw_text:
        cur_seq_letter = last_seq_letter + letter
        if cur_seq_letter in dictionary:
            last_seq_letter = cur_seq_letter
        else:
            lz78_text += f'{dictionary[last_seq_letter]:012b}' + letter
            dictionary[cur_seq_letter] = len(dictionary)
            last_seq_letter = ''

    if last_seq_letter != '':
        lz78_text += f'{dictionary[last_seq_letter]:012b}'

    print(dictionary)

    return lz78_text

In [4]:
def lz78_decompress(lz78_text):

    dictionary = {f'{0:012b}': ''}

    raw_text = ''
    seq_letter = ''

    for idx, letter in enumerate(lz78_text, 1):
        if (idx % 13) == 0:
            raw_text += dictionary[seq_letter] + letter
            dictionary[
                f'{len(dictionary):12b}'] = dictionary[seq_letter] + letter
            seq_letter = ''
        else:
            seq_letter += letter

    if seq_letter != '':
        raw_text += dictionary[seq_letter]

    print(dictionary)

    return raw_text

In [5]:
seq_bin_path = '/content/lz78_algorithm/files/seq.txt'

with open(seq_bin_path) as file:
    data = file.read()
    data = '1011010100010'
    print('Length of read data: {} bits'.format(len(data)))

    compressed = lz78_compress(data)
    print(compressed)
    print('Length of compressed data: {} bits'.format(len(compressed)))
    decompressed = lz78_decompress(compressed)
    print('Length of decompressed data: {} bits'.format(len(decompressed)))

    if data == decompressed:
        print('\nDecompressed data matches compressed data - bingo!\n')

    sum_ones = 0
    for letter in data:
        if letter == '1':
            sum_ones += 1

    print('Sum of ones of original data: {}'.format(sum_ones))
    print('Sum of zeros of original data: {}'.format(len(data) - sum_ones))

NameError: ignored